# Nof users per location

In [86]:
from pyspark.sql import SQLContext
import pandas as pd
import pymysql
import warnings
warnings.filterwarnings("ignore")
from datetime import datetime, timedelta
import logging
from tqdm import tqdm
from fbprophet import Prophet
from sklearn.metrics import mean_squared_error as mse
import math

## Logging ##

import os
import sys


from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
import os
import pandas as pd

from pyspark.sql import SparkSession
import datetime
full_t1 = datetime.datetime.now()
# initialise sparkContext
spark1 = SparkSession.builder \
    .master('local') \
    .appName('p7_sample') \
    .config('spark.executor.memory', '24gb') \
    .config("spark.cores.max", "16") \
    .getOrCreate()

sc = spark1.sparkContext

# using SQLContext to read parquet file
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

def create_prophet_m(app_name,z1,delay=24):
    
    ### --- For realtime pred ---###
    
    full_df = z1.user_count.iloc[0:len(z1)]
    full_df = full_df.reset_index()
    full_df.columns = ['ds','y']
    
    #removing outliers
    q50 = full_df.y.median()
    q100 = full_df.y.quantile(1)
    q75  = full_df.y.quantile(.75)
    #print(max(train_df.y))
    if((q100-q50) >= (2*q75)):
     
        full_df.loc[full_df.y>=(2*q75),'y'] = None
         
    #-- Realtime prediction --##
    #model 
    model_r = Prophet(yearly_seasonality=False,changepoint_prior_scale=.2)
    model_r.fit(full_df)
    future_r = model_r.make_future_dataframe(periods=delay,freq='H')
    forecast_r = model_r.predict(future_r)
    forecast_r.index = forecast_r['ds']
    #forecast 
    pred_r = pd.DataFrame(forecast_r['yhat'][len(z1):(len(z1)+delay)])
    pred_r=pred_r.reset_index()
    #--- completes realtime pred ---#
    
    train_end_index=len(z1.user_count)-delay
    train_df=z1.user_count.iloc[0:train_end_index]
    #train_df= train_df[train_df<cutter]
    
    
    test_df=z1.user_count.iloc[train_end_index:len(z1)]
    
    
    
    train_df=train_df.reset_index()
    test_df=test_df.reset_index()
    train_df.columns=['ds','y']
    
    #--- removing outliers in trainset  ---#
    
    q50 = train_df.y.median()
    q100 = train_df.y.quantile(1)
    q75  = train_df.y.quantile(.75)
    #print(max(train_df.y))
    if((q100-q50) >= (2*q75)):
        
        train_df.loc[train_df.y>=(2*q75),'y'] = None
    
    test_df.columns=['ds','y']
    #print('len of testdf = ',len(test_df))
    #model 
    model = Prophet(yearly_seasonality=False,changepoint_prior_scale=.2)
    model.fit(train_df)
    future = model.make_future_dataframe(periods=len(test_df),freq='H')
    forecast = model.predict(future)
    forecast.index = forecast['ds']
    #forecast 
    pred = pd.DataFrame(forecast['yhat'][train_end_index:len(z1)])
    pred=pred.reset_index()
    pred_df=pd.merge(test_df,pred,on='ds',how='left')
    pred_df.dropna(inplace=True)
    
    df=pd.DataFrame()
    
    if(len(pred_df)>0):
        
        pred_df['error_test']=(pred_df.y-pred_df.yhat)
    
        
    
        MSE=mse(pred_df.y,pred_df.yhat)
        RMSE=math.sqrt(MSE)
        
        pred_df['APE']=abs(pred_df.error_test*100/pred_df.y)
        MAPE=pred_df.APE.mean()
        
        #print("MAPE :",MAPE)
        
        q98=pred_df['APE'].quantile(0.98)
        mape_q98=pred_df['APE'][pred_df.APE<pred_df['APE'].quantile(0.98)].mean()

        df = pd.DataFrame({'length':len(z1),#'predicted_t':[forcast_lag],
                             'test_rmse':RMSE,
                             'test_mape':MAPE,
                 #'test_ape_98':q98,
                 'test_mape_98':mape_q98},
                   
                          index=[app_name])

    return(df,model,forecast,pred_df,pred_r)


#### Reading the datas and merging

In [87]:
import datetime
df = sqlContext.read.parquet('appid_datapoint_parquet1')
df2 =  sqlContext.read.parquet('appid_attribute_parquet')
df2 = df2[['attribute_id','source','target_address','location']]

In [88]:
df.head()

Row(time_stamp=1510801200000, source='134.141.5.104', zone='\\N', target_address='10.51.1.95/', application='Sun RPC', applicationgroup='Protocols', target_attribute_id=215904025, byte_count=8776, tx_byte_count=4920, rx_byte_count=3856, flow_count=59, tx_flow_count=29, rx_flow_count=30, tcp_rsp_time=1011, app_rsp_time=3391, user_data='\\N')

In [89]:
df2.head()

Row(attribute_id=219824640, source='134.141.121.91', target_address='134.141.122.114/maahmed-PC', location='Toronto')

In [90]:
from pyspark.sql.functions import col

df2 = df2.select(col("attribute_id").alias("target_attribute_id"),
                   col("source").alias("source_y"),
                   col("target_address").alias("target_address_y"),
                   col("location").alias("location"), 
                  )
df2.head()

Row(target_attribute_id=219824640, source_y='134.141.121.91', target_address_y='134.141.122.114/maahmed-PC', location='Toronto')

In [91]:
df_merge = df.join(df2,how='left',on='target_attribute_id')
#df_merge.dropna()
#df_merge.head()

In [92]:
t1 = datetime.datetime.now()
data = df_merge.registerTempTable('dummy')
data = sqlContext.sql('select count(distinct target_address) as user_count, time_stamp, location from dummy group by location, time_stamp')

# data cleaning
user_count_df=data.toPandas()
t2 = datetime.datetime.now()
str(t2-t1)

'0:04:20.096422'

In [93]:
pd.to_datetime(pd.Series(user_count_df.time_stamp[50]),unit='ms')

0   2017-11-14 12:00:00
dtype: datetime64[ns]

In [94]:
user_count_df = user_count_df.sort_values(by='location',ascending=True)
dates_outlook = pd.to_datetime(pd.Series(user_count_df.time_stamp),unit='ms')
user_count_df.index = dates_outlook   
user_count_df = user_count_df.sort_values(by='time_stamp')
dates_outlook[0:5]

27954   2018-02-03 23:00:00
6858    2017-12-21 16:00:00
61158   2017-12-08 13:00:00
61159   2018-02-07 06:00:00
2127    2018-01-13 21:00:00
Name: time_stamp, dtype: datetime64[ns]

In [96]:
user_count_df.tail()

,user_count,time_stamp,location
time_stamp,,,
2018-02-09 15:00:00,44,1518188400000,Frankfurt
2018-02-09 15:00:00,9,1518188400000,Paris France
2018-02-09 16:00:00,6236,1518192000000,None
2018-02-09 17:00:00,7754,1518195600000,None
2018-02-09 18:00:00,6286,1518199200000,None


In [10]:
len(user_count_df.location.unique())

43

In [11]:
def forcomb(l,temp):
    
    temp2 = temp[temp.location==l]
    prophet_future_df = pd.DataFrame()
    prophet_analysis_df = pd.DataFrame()
    prophet_df = pd.DataFrame()
        
    if(len(temp2)>1400):
        
        prophet_analysis_df,p_model,p_forcast,prophet_df,prophet_future_df=(create_prophet_m(l,temp2,24))
    
        prophet_future_df['location']=l
    
        prophet_analysis_df['location'] = l
        
        prophet_df['location'] = l

        
    return  prophet_df, prophet_analysis_df, prophet_future_df

In [16]:
l_list = user_count_df.location.unique()
l=l_list[1]
a,b,c = forcomb(l,user_count_df)
b

,length,test_mape,test_mape_98,test_rmse,location
Frankfurt,2170,59.239363,54.835128,138.520684,Frankfurt


In [58]:
from joblib import Parallel, delayed
# Running for all combiantions

qt1 = datetime.datetime.now()

l_list = user_count_df.location.unique()
prophet_df = pd.DataFrame()
prophet_future_df = pd.DataFrame()
prophet_analysis_df = pd.DataFrame()

pool = Parallel(n_jobs=-1,verbose=5,pre_dispatch='all')
r0  = pool(delayed(forcomb)(l,user_count_df) for l in l_list) 

qt2 = datetime.datetime.now()
str(qt2-qt1)

[Parallel(n_jobs=-1)]: Done   8 out of  43 | elapsed:  1.2min remaining:  5.4min
[Parallel(n_jobs=-1)]: Done  17 out of  43 | elapsed:  2.3min remaining:  3.5min
[Parallel(n_jobs=-1)]: Done  26 out of  43 | elapsed:  2.4min remaining:  1.6min
[Parallel(n_jobs=-1)]: Done  35 out of  43 | elapsed:  2.5min remaining:   34.0s
[Parallel(n_jobs=-1)]: Done  43 out of  43 | elapsed:  2.8min finished


'0:02:50.403165'

In [59]:
for i in range(0,len(r0)):
    prophet_df = prophet_df.append(r0[i][0])
    prophet_analysis_df = prophet_analysis_df.append(r0[i][1])
    prophet_future_df = prophet_future_df.append(r0[i][2])
  
 

In [60]:
prophet_analysis_df.sort_values(by='test_mape')

,length,test_mape,test_mape_98,test_rmse,location
Sflow Salem test,1532,7.062800,5.961690,0.630875,Sflow Salem test
RDU,2170,8.976488,6.283665,1030.433805,RDU
Mexico,2157,9.118016,8.011881,20.257379,Mexico
World Wide Router,2170,9.500257,6.485059,1.262258,World Wide Router
Chennai,2170,10.866781,10.490428,94.112444,Chennai
Hong Kong,2108,10.996438,8.478437,0.727973,Hong Kong
Toronto,2170,11.668337,10.179613,183.993264,Toronto
SJ Pleasanton,2148,11.762365,9.590104,29.562050,SJ Pleasanton
Singapore,2146,11.965379,10.299501,6.488914,Singapore
Salem GP,2170,12.182955,11.395482,25.080149,Salem GP


In [98]:
test  = pd.read_csv('p6/user_count_analysis_per_location_data.csv')
test.sort_values(by='test_mape')

,length,test_mape,test_mape_98,test_rmse,location
35,1532,7.062800,5.961690,0.630875,Sflow Salem test
9,2170,8.976488,6.283665,1030.433805,RDU
6,2157,9.118016,8.011881,20.257379,Mexico
30,2170,9.500257,6.485059,1.262258,World Wide Router
16,2170,10.866781,10.490428,94.112444,Chennai
25,2108,10.996438,8.478437,0.727973,Hong Kong
20,2170,11.668337,10.179613,183.993264,Toronto
5,2148,11.762365,9.590104,29.562050,SJ Pleasanton
31,2146,11.965379,10.299501,6.488914,Singapore
0,2170,12.182955,11.395482,25.080149,Salem GP


In [104]:
#for analysis of our model in future

summary_df = pd.DataFrame({'test_mape':[prophet_analysis_df.test_mape.mean()],
                           'test_mape_98':[prophet_analysis_df.test_mape_98.mean()],
                           'test_rmse':[prophet_analysis_df.test_rmse.mean()]},index=[0])
summary_df['run_date'] = datetime.now().date()
summary_df

,test_mape,test_mape_98,test_rmse,run_date
0,19.068324,15.961038,90.028356,2018-07-02


In [20]:
import numpy as np
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)

def plot_test(x,y,title1):
    #label = data.index[0:len(compare_train)]
    label1=x
    #py.init_notebook_mode
        

    # Create a trace
    
    trace1 = go.Scatter(
        x = label1,
        y = np.array(y),
        name ='Actual',
        mode = 'lines+markers'
    
        )


    # Edit the layout
    layout = dict(title = title1,
              xaxis = dict(title = 'Date'),
              yaxis = dict(title = 'app_count'),
              )
    

    data = [trace1]
    fig = dict(data=data, layout=layout)
    fig =go.Figure(data=data, layout=layout)
    py.iplot(fig, filename='basic-line')
    #py.iplot(fig, filename='jupyter/basic_bar')
  

In [21]:
def plot_test1(x,y,y2,title1):
    #label = data.index[0:len(compare_train)]
    label1=x
    #py.init_notebook_mode
        

    # Create a trace
    
    trace1 = go.Scatter(
        x = label1,
        y = np.array(y),
        name ='Actual',
        mode = 'lines+markers'
    
        )
    
    trace2 = go.Scatter(
        x = label1,
        y = np.array(y2),
        name ='Predicted',
        mode = 'lines+markers'
    
        )



    # Edit the layout
    layout = dict(title = title1,
              xaxis = dict(title = 'Date'),
              yaxis = dict(title = 'app_count'),
              )
    

    data = [trace1,trace2]
    fig = dict(data=data, layout=layout)
    fig =go.Figure(data=data, layout=layout)
    py.iplot(fig, filename='basic-line')

In [61]:
l = 'RDU Guest Wireless'
user_count_df_1s = user_count_df[user_count_df.location==l]
plot_test(user_count_df_1s.index,user_count_df_1s.user_count,l)

In [62]:
tt = prophet_df[prophet_df.location==l]
plot_test1(tt.ds,tt.y,tt.yhat,l)

In [65]:
l = 'Madrid'
user_count_df_1s = user_count_df[user_count_df.location==l]
plot_test(user_count_df_1s.index,user_count_df_1s.user_count,l)

In [66]:
tt = prophet_df[prophet_df.location==l]
plot_test1(tt.ds,tt.y,tt.yhat,l)

In [50]:
user_count_df_1s.describe()

,user_count,time_stamp
count,2101.000000,2.101000e+03
mean,5.668729,1.514188e+12
std,13.852238,2.241375e+09
min,1.000000,1.510358e+12
25%,3.000000,1.512270e+12
50%,3.000000,1.514160e+12
75%,4.000000,1.516050e+12
max,128.000000,1.518188e+12


In [51]:
user_count_df_1s.user_count.quantile(.95)

11.0